# Proyecto Curso

Desarrollo del proyecto Final para el Curso de Statistical Learning -  Segundo Trimestre Maestria en Ciencia de Datos- Universidad Galileo.

Este Notebook en específico servirá el Deployment de Modelos ya entrenados para la predicción del Problema "Predicción de Supervivencia Titanic". En un Notebook diferente se hizo el Entrenamiento/Validación de los modelos que aquí se usaran para la predicción.